# SoccerNet SynLoc: Challenge Submission

This notebook covers:
1. Running inference on challenge set
2. Formatting results for submission
3. Creating submission package

## 1. Setup

In [ ]:
import sys
import os
from pathlib import Path

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
    if not os.path.exists('soccernet-synloc'):
        !git clone https://github.com/YOUR_USERNAME/soccernet-synloc.git
        %cd soccernet-synloc
        !pip install -e .[dev] -q
    
    DATA_ROOT = Path('/content/drive/MyDrive/SoccerNet/synloc')
    CHECKPOINT_DIR = Path('/content/drive/MyDrive/SoccerNet/checkpoints')
    SUBMISSION_DIR = Path('/content/drive/MyDrive/SoccerNet/submissions')
else:
    DATA_ROOT = Path('./data/synloc')
    CHECKPOINT_DIR = Path('./checkpoints')
    SUBMISSION_DIR = Path('./submissions')

SUBMISSION_DIR.mkdir(parents=True, exist_ok=True)
print(f"Data root: {DATA_ROOT}")
print(f"Checkpoint dir: {CHECKPOINT_DIR}")
print(f"Submission dir: {SUBMISSION_DIR}")

In [ ]:
import torch
import numpy as np
import json
from torch.utils.data import DataLoader
from datetime import datetime

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")

## 2. Load Model

In [ ]:
from synloc.models import YOLOXPose

# Load config
config_path = CHECKPOINT_DIR / 'config.json'
with open(config_path) as f:
    config = json.load(f)

# Create model
model = YOLOXPose(
    variant=config['model_variant'],
    num_keypoints=config['num_keypoints'],
    input_size=tuple(config['input_size'])
)

# Load weights
checkpoint_path = CHECKPOINT_DIR / 'final_model.pth'
checkpoint = torch.load(checkpoint_path, map_location='cpu')
model.load_state_dict(checkpoint['model_state_dict'])

model = model.to(device)
model.eval()

print(f"Loaded model from epoch {checkpoint.get('epoch', 'unknown')}")

## 3. Prepare Challenge Dataset

In [ ]:
from synloc.data import SynLocDataset, get_val_transforms

# Check challenge set
challenge_ann = DATA_ROOT / 'challenge/annotations.json'
challenge_img_dir = DATA_ROOT / 'challenge/images'

if not challenge_ann.exists():
    print(f"Challenge annotations not found at {challenge_ann}")
    print("Please download the challenge set from SoccerNet")
else:
    # Load annotations to check
    with open(challenge_ann) as f:
        ann_data = json.load(f)
    print(f"Challenge set: {len(ann_data['images'])} images")

In [ ]:
# Create challenge dataset
challenge_dataset = SynLocDataset(
    ann_file=str(challenge_ann),
    img_dir=str(challenge_img_dir),
    transforms=get_val_transforms(config['input_size'][0]),
    input_size=tuple(config['input_size'])
)

challenge_loader = DataLoader(
    challenge_dataset,
    batch_size=config['batch_size'],
    shuffle=False,
    num_workers=4,
    collate_fn=SynLocDataset.collate_fn
)

print(f"Challenge dataset: {len(challenge_dataset)} images")
print(f"Challenge batches: {len(challenge_loader)}")

## 4. Run Inference

In [ ]:
from synloc.evaluation import run_inference

# Use optimized score threshold from validation
# Load from evaluation if available, otherwise use default
SCORE_THRESHOLD = 0.3  # Adjust based on your validation results

# Run inference
print("Running inference on challenge set...")
results = run_inference(
    model,
    challenge_loader,
    device=device,
    score_thr=0.01,  # Low threshold, will filter later
    nms_thr=0.65,
    max_per_img=100
)

print(f"Total detections (raw): {len(results)}")

# Filter by score threshold
filtered_results = [r for r in results if r['score'] >= SCORE_THRESHOLD]
print(f"Total detections (filtered): {len(filtered_results)}")

In [ ]:
# Check detection distribution
from collections import Counter

detections_per_image = Counter(r['image_id'] for r in filtered_results)
counts = list(detections_per_image.values())

print(f"\nDetections per image:")
print(f"  Min: {min(counts) if counts else 0}")
print(f"  Max: {max(counts) if counts else 0}")
print(f"  Mean: {np.mean(counts) if counts else 0:.1f}")
print(f"  Images with 0 detections: {len(challenge_dataset) - len(detections_per_image)}")

## 5. Format Submission

In [ ]:
from synloc.evaluation import format_results_for_submission, create_submission_zip

# Create submission directory with timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
submission_name = f"submission_{timestamp}"
submission_path = SUBMISSION_DIR / submission_name
submission_path.mkdir(parents=True, exist_ok=True)

# Format results
results_path, metadata_path = format_results_for_submission(
    results=filtered_results,
    score_threshold=SCORE_THRESHOLD,
    position_from_keypoint_index=1,  # pelvis_ground
    output_dir=str(submission_path)
)

print(f"Created:")
print(f"  {results_path}")
print(f"  {metadata_path}")

In [ ]:
# Verify results format
with open(results_path) as f:
    saved_results = json.load(f)

print(f"\nResults file:")
print(f"  Number of detections: {len(saved_results)}")

if len(saved_results) > 0:
    print(f"  Sample detection:")
    sample = saved_results[0]
    for k, v in sample.items():
        if isinstance(v, list) and len(v) > 5:
            print(f"    {k}: list[{len(v)}]")
        else:
            print(f"    {k}: {v}")

with open(metadata_path) as f:
    metadata = json.load(f)

print(f"\nMetadata file:")
for k, v in metadata.items():
    print(f"  {k}: {v}")

## 6. Create Submission Zip

In [ ]:
# Create zip file
zip_path = create_submission_zip(
    results_path=results_path,
    metadata_path=metadata_path,
    output_path=str(SUBMISSION_DIR / f"{submission_name}.zip")
)

print(f"Created submission zip: {zip_path}")

# Check zip size
import os
zip_size = os.path.getsize(zip_path) / (1024 * 1024)
print(f"Zip size: {zip_size:.2f} MB")

In [ ]:
# Verify zip contents
import zipfile

with zipfile.ZipFile(zip_path, 'r') as zf:
    print("Zip contents:")
    for info in zf.infolist():
        print(f"  {info.filename}: {info.file_size:,} bytes")

## 7. Visualize Sample Predictions

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
from synloc.evaluation import visualize_predictions
from synloc.visualization import draw_pitch
from synloc.data.camera import keypoint_to_world

def visualize_challenge_prediction(img_id, results, data_root, ann_data, threshold=0.3):
    """Visualize predictions for a challenge image."""
    # Get image info
    img_info = next(img for img in ann_data['images'] if img['id'] == img_id)
    
    # Get predictions
    preds = [r for r in results if r['image_id'] == img_id and r['score'] >= threshold]
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Image view
    img_path = data_root / 'challenge/images' / img_info['file_name']
    img = Image.open(img_path)
    axes[0].imshow(img)
    
    # Draw predictions
    for pred in preds:
        x, y, w, h = pred['bbox']
        rect = plt.Rectangle((x, y), w, h, fill=False, 
                              edgecolor='green', linewidth=2)
        axes[0].add_patch(rect)
        
        # Draw keypoints
        kpts = np.array(pred['keypoints']).reshape(-1, 3)
        axes[0].scatter(kpts[0, 0], kpts[0, 1], c='red', s=30, zorder=10)  # pelvis
        axes[0].scatter(kpts[1, 0], kpts[1, 1], c='blue', s=30, zorder=10)  # pelvis_ground
    
    axes[0].set_title(f"Image {img_id}: {len(preds)} detections")
    axes[0].axis('off')
    
    # BEV view
    camera_matrix = torch.tensor(img_info['camera_matrix'], dtype=torch.float32)
    undist_poly = torch.tensor(img_info['undist_poly'], dtype=torch.float32)
    
    positions = []
    for pred in preds:
        kpts = np.array(pred['keypoints']).reshape(-1, 3)
        kpt = torch.tensor(kpts[1, :2], dtype=torch.float32).unsqueeze(0)
        kpt_norm = (kpt - torch.tensor([(img_info['width']-1)/2, (img_info['height']-1)/2])) / img_info['width']
        
        try:
            world = keypoint_to_world(camera_matrix, undist_poly, kpt_norm)
            positions.append(world[0, :2].numpy())
        except:
            pass
    
    if positions:
        positions = np.array(positions)
        axes[1] = draw_pitch(ax=axes[1])
        axes[1].scatter(positions[:, 0], positions[:, 1],
                       c='red', s=100, marker='o',
                       edgecolors='white', linewidths=2,
                       zorder=10)
    else:
        axes[1] = draw_pitch(ax=axes[1])
    
    axes[1].set_title('BEV Projection')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Visualize random samples
np.random.seed(42)
sample_ids = np.random.choice(
    [img['id'] for img in ann_data['images']], 
    size=min(4, len(ann_data['images'])), 
    replace=False
)

for img_id in sample_ids:
    visualize_challenge_prediction(img_id, filtered_results, DATA_ROOT, ann_data, SCORE_THRESHOLD)

## 8. Submission Instructions

In [ ]:
print("="*60)
print("SUBMISSION READY")
print("="*60)
print(f"\nSubmission file: {zip_path}")
print(f"Detections: {len(filtered_results)}")
print(f"Score threshold: {SCORE_THRESHOLD}")
print(f"Keypoint index: 1 (pelvis_ground)")
print()
print("To submit:")
print("1. Go to https://eval.ai/web/challenges/challenge-page/XXX")
print("2. Select 'SynLoc' phase")
print(f"3. Upload: {Path(zip_path).name}")
print()
print("Good luck!")

## 9. Optional: Test-Time Augmentation

In [ ]:
def run_tta_inference(model, dataloader, device='cuda', scales=[1.0], flip=True):
    """Run inference with test-time augmentation.
    
    Args:
        model: YOLOX-Pose model.
        dataloader: Data loader.
        device: Device.
        scales: List of scales to use.
        flip: Whether to use horizontal flip.
    
    Returns:
        List of results.
    """
    from tqdm.auto import tqdm
    import torch.nn.functional as F
    
    model.eval()
    model = model.to(device)
    
    all_results = []
    det_id = 1
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc='TTA Inference'):
            images = batch['image'].to(device)
            img_ids = batch['img_id']
            orig_sizes = batch['orig_size']
            
            _, _, h, w = images.shape
            
            # Collect predictions from all augmentations
            all_preds = []
            
            for scale in scales:
                if scale != 1.0:
                    new_h, new_w = int(h * scale), int(w * scale)
                    scaled = F.interpolate(images, (new_h, new_w), mode='bilinear')
                else:
                    scaled = images
                    new_h, new_w = h, w
                
                # Original
                results = model.predict(scaled, input_size=(new_w, new_h), score_thr=0.01)
                all_preds.append((results, 1.0, False))
                
                # Flipped
                if flip:
                    flipped = torch.flip(scaled, dims=[-1])
                    results_flip = model.predict(flipped, input_size=(new_w, new_h), score_thr=0.01)
                    all_preds.append((results_flip, scale, True))
            
            # Merge predictions (simple NMS-based merge)
            # For now, just use original predictions
            # You can implement more sophisticated merging
            results_batch = all_preds[0][0]
            
            # Format results (same as run_inference)
            for img_id, orig_size, results in zip(img_ids, orig_sizes, results_batch):
                scale_x = orig_size[0] / w
                scale_y = orig_size[1] / h
                
                bboxes = results['bboxes'].cpu().numpy()
                scores = results['scores'].cpu().numpy()
                keypoints = results['keypoints'].cpu().numpy()
                kpt_scores = results['keypoint_scores'].cpu().numpy()
                
                for i in range(len(bboxes)):
                    x1, y1, x2, y2 = bboxes[i]
                    x1, x2 = x1 * scale_x, x2 * scale_x
                    y1, y2 = y1 * scale_y, y2 * scale_y
                    bbox_w, bbox_h = x2 - x1, y2 - y1
                    
                    kpts = keypoints[i].copy()
                    kpts[:, 0] *= scale_x
                    kpts[:, 1] *= scale_y
                    
                    kpts_flat = []
                    for k in range(len(kpts)):
                        kpts_flat.extend([
                            float(kpts[k, 0]),
                            float(kpts[k, 1]),
                            float(kpt_scores[i, k])
                        ])
                    
                    result = {
                        'id': det_id,
                        'image_id': int(img_id),
                        'category_id': 1,
                        'bbox': [float(x1), float(y1), float(bbox_w), float(bbox_h)],
                        'area': float(bbox_w * bbox_h),
                        'score': float(scores[i]),
                        'keypoints': kpts_flat,
                    }
                    all_results.append(result)
                    det_id += 1
    
    return all_results

print("TTA function defined. Uncomment below to use.")

In [ ]:
# Uncomment to run with TTA
# tta_results = run_tta_inference(
#     model,
#     challenge_loader,
#     device=device,
#     scales=[1.0],  # Add more scales: [0.8, 1.0, 1.2]
#     flip=True
# )
# print(f"TTA results: {len(tta_results)} detections")

## Summary

Submission package created successfully!

Files generated:
- `results.json`: Detection results in COCO format
- `metadata.json`: Score threshold and keypoint index
- `submission_YYYYMMDD_HHMMSS.zip`: Ready for upload

Model settings used:
- Variant: {config['model_variant']}
- Input size: {config['input_size']}
- Score threshold: {SCORE_THRESHOLD}